# Benchmark dataloders of Tahoe100M dataset

In [ ]:
from __future__ import annotations

!pip install -q lamindb
!pip install -q -e /home/sagemaker-user/annbatch
!pip install -q -e /home/sagemaker-user/arrayloader-benchmarks
!pip install -q zarrs
!pip install -q scDataset

In [1]:
import json

In [2]:
import lamindb as ln

ln.track(project="zjQ6EYzMXif4")

→ connected lamindb: laminlabs/arrayloader-benchmarks
→ found notebook run_benchmarks.ipynb, making new version
→ created Transform('e6Ry7noc4Y0d0004', key='run_benchmarks.ipynb'), started new Run('CVMhkZeKFU1XX4RO') at 2025-10-30 08:49:55 UTC
→ notebook imports: lamindb==1.15.0 matplotlib==3.10.5 pandas==2.3.1 seaborn==0.13.2
• recommendation: to identify the notebook across renames, pass the uid: ln.track("e6Ry7noc4Y0d", project="zjQ6EYzMXif4")


## Arrayloaders

In [3]:
%%capture cap --no-stderr
!python annbatch/benchmark_annbatch.py --chunk_size=64 --preload_nchunks=128 --use_torch_loader=False --n_samples=2000000

In [4]:
arrayloaders_64 = json.loads(cap.stdout.split("\n")[-2].rstrip())["samples/sec"]  # noqa: F821
arrayloaders_64

22970.167872797523

In [5]:
%%capture cap --no-stderr
!python annbatch/benchmark_annbatch.py --chunk_size=128 --preload_nchunks=64 --use_torch_loader=False --n_samples=2000000

In [6]:
arrayloaders_128 = json.loads(cap.stdout.split("\n")[-2].rstrip())["samples/sec"]  # noqa: F821
arrayloaders_128

26921.01644485809

In [7]:
%%capture cap --no-stderr
!python annbatch/benchmark_annbatch.py --chunk_size=256 --preload_nchunks=32 --use_torch_loader=False --n_samples=2000000

In [8]:
arrayloaders_256 = json.loads(cap.stdout.split("\n")[-2].rstrip())["samples/sec"]  # noqa: F821
arrayloaders_256

29921.343281593807

In [9]:
%%capture cap --no-stderr
!python annbatch/benchmark_annbatch.py --chunk_size=512 --preload_nchunks=16 --use_torch_loader=False --n_samples=2000000

In [10]:
arrayloaders_512 = json.loads(cap.stdout.split("\n")[-2].rstrip())["samples/sec"]  # noqa: F821
arrayloaders_512

32149.175057491768

## MappedCollection

In [11]:
%%capture cap --no-stderr
!python mapped_collection/benchmark_mapped_collection.py

In [12]:
mapped_collection = json.loads(cap.stdout.split("\n")[-2].rstrip())["samples/sec"]  # noqa: F821
mapped_collection

1085.4687136075536

## scDataset

In [13]:
%%capture cap --no-stderr
!python scDataset/benchmark_scDataset.py --num_workers=8 --batch_size=64  # use small batch size - otherwise dataloader is slow

In [14]:
sc_dataset = json.loads(cap.stdout.split("\n")[-2].rstrip())["samples/sec"]  # noqa: F821
sc_dataset

In [15]:
%%capture cap --no-stderr
!python scDataset/benchmark_scDataset.py --num_workers=8 --batch_size=4096 --block_size=256 --fetch_factor=2

In [16]:
sc_dataset_matched = json.loads(cap.stdout.split("\n")[-2].rstrip())["samples/sec"]  # noqa: F821
sc_dataset_matched

23048.011507048956

## Plot results

In [18]:
import pandas as pd

results = pd.DataFrame(
    {
        "loader": [
            "annbatch (chunk_size=64)",
            "annbatch (chunk_size=128)",
            "annbatch (chunk_size=256)",
            "annbatch (chunk_size=512)",
            "MappedCollection",
            "scDataset (recommended settings)",
            "scDataset (annbatch matched settings)",
        ],
        "samples/sec": [
            arrayloaders_64,
            arrayloaders_128,
            arrayloaders_256,
            arrayloaders_512,
            mapped_collection,
            sc_dataset,
            sc_dataset_matched,
        ],
    }
)
results

,loader,samples/sec
0,annbatch (chunk_size=64),22970.167873
1,annbatch (chunk_size=128),26921.016445
2,annbatch (chunk_size=256),29921.343282
3,annbatch (chunk_size=512),32149.175057
4,MappedCollection,1085.468714
5,scDataset (recommended settings),2112.048406
6,scDataset (annbatch matched settings),23048.011507


In [19]:
ln.Artifact.from_dataframe(
    results,
    key="dataloader_v2_results_benchmark.parquet",
    description="Results of the dataloder v2 benchmarks",
).save()

→ writing the in-memory object into cache
→ creating new artifact version for key 'dataloader_v2_results_benchmark.parquet' in storage 's3://lamin-us-west-2/wXDsTYYd'
... uploading p2NK2Cs2lbw72vVc0003.parquet: 100.0%
• replacing the existing cache path /home/sagemaker-user/.cache/lamindb/lamin-us-west-2/wXDsTYYd/dataloader_v2_results_benchmark.parquet


Artifact(uid='p2NK2Cs2lbw72vVc0003', version=None, is_latest=True, key='dataloader_v2_results_benchmark.parquet', description='Results of the dataloder v2 benchmarks', suffix='.parquet', kind='dataset', otype='DataFrame', size=2368, hash='XoNgc3887Zyzw7tfocQU7g', n_files=None, n_observations=7, branch_id=1, space_id=1, storage_id=3, run_id=347, schema_id=None, created_by_id=4, created_at=2025-10-30 10:11:58 UTC, is_locked=False)

In [20]:
ln.finish()

• please hit CTRL + s to save the notebook in your editor .... still waiting ... ✓
! cells [(0, None), (None, 1), (16, 18)] were not run consecutively
→ finished Run('CVMhkZeKFU1XX4RO') after 1h at 2025-10-30 10:12:10 UTC
→ go to: https://lamin.ai/laminlabs/arrayloader-benchmarks/transform/e6Ry7noc4Y0d0004
→ to update your notebook from the CLI, run: lamin save /home/sagemaker-user/arrayloader-benchmarks/src/arrayloader_benchmarks/run_benchmarks.ipynb
